In [4]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\HAL 9004\\.conda\\envs\\spark-polito\\lib\\site-packages\\pyspark'

In [ ]:
#Exercise 37
# Maximum values
# Input: a textual csv file containing the daily value of PM10 for a set of sensors
#▪ Each line of the files has the following format
#sensorId,date,PM10 value (μg/m3)\n 
# Output: the maximum value of PM10 for each sensor
#▪ Store the result in an HDFS file


## With RDD's

In [5]:
#entry point for RDD's
from pyspark import SparkContext

sc = SparkContext("local", "Excercise 37")

In [23]:
rdd_37 = sc.textFile("input_file_37.txt")

In [24]:
rdd_37.take(30)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5']

In [25]:
rdd_37 = rdd_37.map(lambda a: a.split(","))

In [26]:
rdd_37.take(30)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [27]:
rdd_37_proc = rdd_37.map(lambda a: (a[0],a[2]))

In [28]:
rdd_37_proc.take(10)

[('s1', '20.5'),
 ('s2', '30.1'),
 ('s1', '60.2'),
 ('s2', '20.4'),
 ('s1', '55.5'),
 ('s2', '52.5')]

In [29]:
rdd_37_proc = rdd_37_proc.reduceByKey(max)

In [30]:
rdd_37_proc.take(10)

[('s1', '60.2'), ('s2', '52.5')]

In [31]:
rdd_37.take(10)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [35]:
rdd_37_proc_2 = rdd_37

In [34]:
print(id(rdd_37_proc_2),id(rdd_37))

2281375524224 2281375524224


In [36]:
rdd_37_proc_2 = rdd_37.map(lambda a: [a[0],a[1],a[2]])

In [37]:
print(id(rdd_37_proc_2),id(rdd_37))

2281367626128 2281375524224


In [41]:
rdd_37_proc_2.map( lambda a:float(a[2]) ).reduce(max)

60.2

In [71]:
rdd_38 = sc.textFile("input_file_37.txt")
rdd_38.take(10)

['s1,2016-01-01,20.5',
 's2,2016-01-01,30.1',
 's1,2016-01-02,60.2',
 's2,2016-01-02,20.4',
 's1,2016-01-03,55.5',
 's2,2016-01-03,52.5']

In [72]:
rdd_38 = rdd_38.map(lambda y:y.split(','))
rdd_38.take(10)

[['s1', '2016-01-01', '20.5'],
 ['s2', '2016-01-01', '30.1'],
 ['s1', '2016-01-02', '60.2'],
 ['s2', '2016-01-02', '20.4'],
 ['s1', '2016-01-03', '55.5'],
 ['s2', '2016-01-03', '52.5']]

In [73]:
rdd_38 = rdd_38.map(lambda h: (h[0],h[1],float(h[2])  ))
rdd_38.take(10)

[('s1', '2016-01-01', 20.5),
 ('s2', '2016-01-01', 30.1),
 ('s1', '2016-01-02', 60.2),
 ('s2', '2016-01-02', 20.4),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

In [75]:
rdd_38_res = rdd_38.filter(lambda h: h[2]>50)
rdd_38_res.take(10)

[('s1', '2016-01-02', 60.2),
 ('s1', '2016-01-03', 55.5),
 ('s2', '2016-01-03', 52.5)]

### With DataFrames

In [42]:
#entry point for other API's
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Excercise 37") \
    .getOrCreate()

In [44]:
rdd_37 = spark.read.text("input_file_37.txt")
rdd_37.take(10)

[Row(value='s1,2016-01-01,20.5'),
 Row(value='s2,2016-01-01,30.1'),
 Row(value='s1,2016-01-02,60.2'),
 Row(value='s2,2016-01-02,20.4'),
 Row(value='s1,2016-01-03,55.5'),
 Row(value='s2,2016-01-03,52.5')]

In [ ]:
#TODO WITH DATAFRAMES

## With SQL

In [45]:
rdd_37 = sc.textFile("input_file_37.txt")
rdd_37 = rdd_37.map(lambda a: a.split(","))
rdd_36 = rdd_37.map(lambda a: (a[0],a[1],float(a[2])) )
df_aux =rdd_37.toDF()



In [46]:
df_aux.createOrReplaceTempView("sql37")


In [47]:
sqlDF = spark.sql("SELECT * FROM sql37")
sqlDF.show()

+---+----------+----+
| _1|        _2|  _3|
+---+----------+----+
| s1|2016-01-01|20.5|
| s2|2016-01-01|30.1|
| s1|2016-01-02|60.2|
| s2|2016-01-02|20.4|
| s1|2016-01-03|55.5|
| s2|2016-01-03|52.5|
+---+----------+----+



In [51]:
sqlDF = spark.sql("SELECT MAX(_3) FROM sql37 GROUP BY _1")
sqlDF.show()

+-------+
|max(_3)|
+-------+
|   52.5|
|   60.2|
+-------+



In [ ]:
sqlDF

In [ ]:
spark.sql("SELECT * FROM sql36").show()

In [ ]:
spark.sql("SELECT *, cast(_3 as float) FROM sql36").show()